In [3]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
from multiprocessing.dummy import Pool

# 定义函数遍历文件夹中所有PNG文件
def find_png_files(folder_path):
    png_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.png'):
                png_files.append(os.path.join(root, file))
    return png_files

# 定义函数计算单张图片的均值和方差
def calculate_mean_and_variance_for_image(path):
    mean = 0
    std = 0
    png_files = find_png_files(path)
    for img in png_files:
        # 读取图片
        image = Image.open(img)
        image = np.asarray(image)
        mean += np.mean(image)
        std += np.std(image)
    
    try: 
        mean /= len(png_files)
        std /= len(png_files)
        return mean, std
    except Exception as e:
        print(f"Error {e}: {path}")
        return 0, 0
    

# 定义函数计算均值和方差
def calculate_mean_and_variance(paths):
    # 初始化均值和方差
    mean = np.zeros(1)
    variance = np.zeros(1)
    process_num = 40
    pool = Pool(process_num)
    results = []

    # 找到路径下所有文件夹名
    folders = os.listdir(paths)
    num_folders = len(folders)

    for folder in folders:
        folder = os.path.join(paths, folder)
        results.append(pool.apply_async(calculate_mean_and_variance_for_image, (folder,)))
    pool.close()

    # 将每张图片的均值和方差累加起来
    for i, result in tqdm(enumerate(results),total=num_folders):
        mean += result.get()[0]
        variance += result.get()[1]
    
    pool.join()

    # 计算总体均值和方差
    mean /= num_folders
    variance /= num_folders
    
    return mean, variance



In [4]:
root_path = r'D:\PostGraduate\DL\mgam_CT\data\2023_Med_CQK\img'
mean, variance = calculate_mean_and_variance(root_path)
print('Mean:', mean)
print('Variance:', variance)

  0%|          | 0/648 [00:00<?, ?it/s]

100%|██████████| 648/648 [08:02<00:00,  1.34it/s]

Mean: [29540.09186308]
Variance: [16727.8130729]


In [8]:
from PIL import Image

def get_channels(image_path):
    with Image.open(image_path) as img:
        return img.mode

image_path = r"D:\PostGraduate\DL\mgam_CT\data\2023_Med_CQK\img\柏家荣\ImageFileName000.png"
channels = get_channels(image_path)
print(f'The image at {image_path} has {len(channels)} channel(s): {channels}')


The image at D:\PostGraduate\DL\mgam_CT\data\2023_Med_CQK\img\柏家荣\ImageFileName000.png has 1 channel(s): I


In [2]:
from PIL import Image
import numpy as np

path = r"D:\PostGraduate\DL\mgam_CT\data\2023_Med_CQK\img\柏家荣\ImageFileName000.png"
image = Image.open(path)
image = np.asarray(image)
print(image.max(),image.min(),image.mean(),image.std())


65535 0 31566.25786590576 17640.927570253447
